In [13]:
import vtk
import nibabel as nib
from skimage import measure
import imageio
import os

def load_mesh_as_vtk(filename):
    """Load NIfTI file and convert to a VTK PolyData object."""
    img = nib.load(filename)
    data = img.get_fdata()

    # Generate vertices and faces from binary data using marching cubes
    verts, faces, _, _ = measure.marching_cubes(data, level=0.5)

    # Create a VTK PolyData object
    vtk_points = vtk.vtkPoints()
    for vert in verts:
        vtk_points.InsertNextPoint(vert)

    vtk_faces = vtk.vtkCellArray()
    for face in faces:
        vtk_faces.InsertNextCell(3)
        for i in face:
            vtk_faces.InsertCellPoint(int(i))

    mesh = vtk.vtkPolyData()
    mesh.SetPoints(vtk_points)
    mesh.SetPolys(vtk_faces)
    
    return mesh

def visualize_mesh_overlap_vtk(mesh1, mesh2, output_file):
    """Visualize the overlap between two VTK meshes with a 90-degree left rotation and save as .png."""
    # Create a mapper and actor for mesh1 (Ground Truth)
    mapper1 = vtk.vtkPolyDataMapper()
    mapper1.SetInputData(mesh1)
    actor1 = vtk.vtkActor()
    actor1.SetMapper(mapper1)
    actor1.GetProperty().SetColor(1, 1, 0)  # Yellow for Ground Truth
    actor1.GetProperty().SetOpacity(0.5)

    # Create a mapper and actor for mesh2 (BBDM Prediction)
    mapper2 = vtk.vtkPolyDataMapper()
    mapper2.SetInputData(mesh2)
    actor2 = vtk.vtkActor()
    actor2.SetMapper(mapper2)
    actor2.GetProperty().SetColor(1, 0, 0)  # Red for BBDM Prediction
    actor2.GetProperty().SetOpacity(0.5)

    # Apply a 90-degree left rotation to both actors around the Z-axis
    transform = vtk.vtkTransform()
    transform.RotateZ(-90)  # Rotate -90 degrees around Z-axis (to the left)

    actor1.SetUserTransform(transform)
    actor2.SetUserTransform(transform)

    # Renderer
    renderer = vtk.vtkRenderer()
    renderer.AddActor(actor1)
    renderer.AddActor(actor2)
    renderer.SetBackground(1, 1, 1)  # White background

    # Render window
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window.SetOffScreenRendering(1)  # Enable offscreen rendering for saving to file

    # Render the scene
    render_window.Render()

    # Save the image as a .png file
    window_to_image_filter = vtk.vtkWindowToImageFilter()
    window_to_image_filter.SetInput(render_window)
    window_to_image_filter.Update()

    writer = vtk.vtkPNGWriter()
    writer.SetFileName(output_file)
    writer.SetInputConnection(window_to_image_filter.GetOutputPort())
    writer.Write()

def process_scan(scan_number):
    """Process and visualize all the scans for a given scan number."""
    # Create output directory structure
    output_dir = f'visualizations/scan_{scan_number}'
    os.makedirs(output_dir, exist_ok=True)

    image_filenames = []

    # Loop over the 15 files and save images
    for i in range(1, 16):
        file_suffix = f'{i:02d}'  # Format number with leading zeros, e.g., '01', '02', ..., '15'
        gt_file = f'gt/scan_{scan_number}_b{file_suffix}.nii.gz'       # Ground Truth
        pred_file = f'pred_BBDM/scan_{scan_number}_b{file_suffix}.nii.gz' # Prediction

        # Load the NIfTI files as VTK PolyData objects
        mesh1 = load_mesh_as_vtk(gt_file)
        mesh2 = load_mesh_as_vtk(pred_file)

        # Save the visualization as a .png file with the scan number in the name
        output_filename = os.path.join(output_dir, f'scan_{scan_number}_b{file_suffix}.png')
        visualize_mesh_overlap_vtk(mesh1, mesh2, output_file=output_filename)
        image_filenames.append(output_filename)

    # Create a video from the images
    video_filename = os.path.join(output_dir, f'scan_{scan_number}_video.mp4')
    with imageio.get_writer(video_filename, fps=1) as writer:  # Adjust fps as needed
        for filename in image_filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    print(f"Video saved as {video_filename}")

# Example usage
process_scan(224)  # Change the scan number as needed


/var/folders/75/cyqnjl4d39nbnmv8m5ttxbf40000gn/T/ipykernel_13095/2169240095.py:108: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (300, 300) to (304, 304) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
[swscaler @ 0x7fa950090000] Warning: data is not aligned! This can lead to a speed loss


Video saved as visualizations/scan_224/scan_224_video.mp4


: 